<a href="https://colab.research.google.com/github/joaodpcm/MDA/blob/data_exploration_david/preprocessing_data_mda_david.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Exploration**

---






# Importing Packages


In [ ]:
# Importing packages 
import pandas as pd
import numpy as np
import seaborn as sn
import os

# Clone repository

In [ ]:
# Repository path
repo_path = 'https://github.com/joaodpcm/MDAgit'
#Accessing GitHub repository
!git clone <https://github.com/joaodpcm/MDAgit>

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `git clone <https://github.com/joaodpcm/MDAgit>'


# Loading data